![Ironhack logo](https://i.imgur.com/1QgrNNw.png)

<body>
    <p style="font-size:28px;text-align:center"><b>Project 03 - Part 02 | Data Cleaning</b></p>
</body>

# Introduction

The objective of the second part of this project was clean the data collected, so it can be analyzed in the software Tableau.

---

<body>
    <p style="font-size:20px"><b>Problem Statement</b></p>
</body>

_Do some TikTok viral songs have common characteristics?_


# Setup

## Import

In [1]:
import os
import re

from ast import literal_eval

import sqlalchemy as db
import numpy as np
import pandas as pd

from dotenv import load_dotenv, find_dotenv

# Songs

## Import datasets

When the dataset is imported and stored in a Pandas DataFrame, the columns that were a result of a request to an API become strings, so the values in these columns need to be converted to dictionaries.

### Spotify

In [2]:
# Import the dataset
df_sp_songs = pd.read_csv('exported_df/sp_tracks_info.csv')

# Remove unnecessary columns
df_sp_songs = df_sp_songs.drop(columns=['spotify_search', 'sp_id'])

# Check the result
df_sp_songs

,song,artists,artists_list,number_artists,sp_name,sp_duration_ms,sp_popularity,sp_release_date,sp_explicit,sp_artists_name,...,sp_key,sp_loudness,sp_mode,sp_speechiness,sp_acousticness,sp_instrumentalness,sp_liveness,sp_valence,sp_tempo,sp_time_signature
0,Roxanne,Arizona Zervas,['Arizona Zervas'],1,ROXANNE,163636,88,2019-10-10,True,['Arizona Zervas'],...,6,-5.616,0,0.1480,0.05220,0.000000,0.4600,0.4570,116.735,5
1,Say So,Doja Cat,['Doja Cat'],1,Say So,237893,88,2019-11-07,True,['Doja Cat'],...,11,-4.577,0,0.1580,0.25600,0.000004,0.0904,0.7860,110.962,4
2,My Oh My,Camila Cabello feat. DaBaby,"['Camila Cabello', 'DaBaby']",2,My Oh My (feat. DaBaby),170746,82,2019-12-06,False,"['Camila Cabello', 'DaBaby']",...,8,-6.024,1,0.0296,0.01800,0.000013,0.0887,0.3830,105.046,4
3,Moon,Kid Francescoli,['Kid Francescoli'],1,Moon (And It Went Like),390638,70,2017-03-03,False,['Kid Francescoli'],...,7,-10.002,1,0.0345,0.28800,0.856000,0.1020,0.0584,117.986,4
4,Vibe,Cookiee Kawaii,['Cookiee Kawaii'],1,Vibe (If I Back It Up),83940,72,2019-03-29,True,['Cookiee Kawaii'],...,10,-8.719,1,0.3440,0.06350,0.009320,0.1180,0.1750,159.947,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,What the Hell,Avril Lavigne,['Avril Lavigne'],1,What the Hell,220706,74,2011-03-08,False,['Avril Lavigne'],...,6,-3.689,0,0.0548,0.00472,0.012700,0.1400,0.8770,149.976,4
62,Towards the Sun,Rihanna,['Rihanna'],1,"Towards The Sun - From The ""Home"" Soundtrack",273293,55,2015-03-23,False,['Rihanna'],...,4,-6.207,0,0.0392,0.05310,0.000000,0.1520,0.2630,170.180,4
63,I Think I'm OKAY,"Machine Gun Kelly, YUNGBLUD, and Travis Barker","['Machine Gun Kelly', 'YUNGBLUD', 'Travis Bark...",3,I Think I'm OKAY (with YUNGBLUD & Travis Barker),169397,81,2019-07-05,True,"['Machine Gun Kelly', 'YUNGBLUD', 'Travis Bark...",...,7,-4.718,1,0.0379,0.02570,0.000000,0.3130,0.2770,119.921,4
64,Myself,Bazzi,['Bazzi'],1,Myself,167552,76,2018-04-12,False,['Bazzi'],...,9,-5.513,0,0.0720,0.46500,0.000001,0.0338,0.9020,195.918,4


In [3]:
# Convert some columns from strings to dictionaries
df_sp_songs.artists_list = df_sp_songs.artists_list.apply(literal_eval)
df_sp_songs.sp_artists_name = df_sp_songs.sp_artists_name.apply(literal_eval)

In [4]:
# Check the result
print(f'Column "artists_list": {type(df_sp_songs.artists_list[0])}')
print(f'Column "sp_artists_name": {type(df_sp_songs.sp_artists_name[0])}')

Column "artists_list": <class 'list'>
Column "sp_artists_name": <class 'list'>


### Chartmetric

In [5]:
# Import the dataset
df_cmc_songs = pd.read_csv('exported_df/cmc_tracks_info.csv')

# Select only the songs that were found in Spotify
df_cmc_songs = df_cmc_songs[df_cmc_songs.cmc_id != 'not-found']

# Check the result
df_cmc_songs

,song,artists,artists_list,number_artists,sp_id,cmc_song_seach,cmc_id,cmc_track_metadata,cmc_track_stats_tiktok
0,Roxanne,Arizona Zervas,['Arizona Zervas'],1,696DnlkuDOXcMAnKlTgXXK,"{'obj': {'tracks': [{'id': 27228348, 'name': '...",27228348,"{'obj': {'id': 27228348, 'name': 'ROXANNE', 'i...","{'obj': [{'value': 2100000, 'timestp': '2020-0..."
1,Say So,Doja Cat,['Doja Cat'],1,3Dv1eDb0MEgF93GpLXlucZ,"{'obj': {'tracks': [{'id': 26951096, 'name': '...",26951096,"{'obj': {'id': 26951096, 'name': 'Say So', 'is...","{'obj': [{'value': 5300000, 'timestp': '2020-0..."
2,My Oh My,Camila Cabello feat. DaBaby,"['Camila Cabello', 'DaBaby']",2,3yOlyBJuViE2YSGn3nVE1K,"{'obj': {'tracks': [{'id': 27597895, 'name': '...",27597895,"{'obj': {'id': 27597895, 'name': 'My Oh My', '...","{'obj': [{'value': 791700, 'timestp': '2020-01..."
3,Moon,Kid Francescoli,['Kid Francescoli'],1,24upABZ8A0sAepfu91sEYr,"{'obj': {'tracks': [{'id': 12263271, 'name': '...",12263271,"{'obj': {'id': 12263271, 'name': 'Moon', 'isrc...","{'obj': [{'value': 382197, 'timestp': '2020-02..."
4,Vibe,Cookiee Kawaii,['Cookiee Kawaii'],1,4gOgQTv9RYYFZ1uQNnlk3q,"{'obj': {'tracks': [{'id': 25138356, 'name': '...",25138356,"{'obj': {'id': 25138356, 'name': 'Vibe', 'isrc...","{'obj': [{'value': 948913, 'timestp': '2020-03..."
...,...,...,...,...,...,...,...,...,...
64,What the Hell,Avril Lavigne,['Avril Lavigne'],1,2z4U9d5OAA4YLNXoCgioxo,"{'obj': {'tracks': [{'id': 15674355, 'name': '...",15674355,"{'obj': {'id': 15674355, 'name': 'What The Hel...","{'obj': [{'value': 315800, 'timestp': '2020-02..."
65,Towards the Sun,Rihanna,['Rihanna'],1,1UuZhGTon3gzXQAJzNa2A4,"{'obj': {'tracks': [{'id': 13795060, 'name': '...",13795060,"{'obj': {'id': 13795060, 'name': 'Towards The ...","{'obj': [{'value': 0, 'timestp': '2020-01-07T0..."
66,I Think I'm OKAY,"Machine Gun Kelly, YUNGBLUD, and Travis Barker","['Machine Gun Kelly', 'YUNGBLUD', 'Travis Bark...",3,2gTdDMpNxIRFSiu7HutMCg,"{'obj': {'tracks': [{'id': 23903635, 'name': ""...",23903635,"{'obj': {'id': 23903635, 'name': ""I Think I'm ...","{'obj': [{'value': 60100, 'timestp': '2020-01-..."
67,Myself,Bazzi,['Bazzi'],1,5YLHLxoZsodDWjqSgjhBf3,"{'obj': {'tracks': [{'id': 19040153, 'name': '...",19040153,"{'obj': {'id': 19040153, 'name': 'Myself', 'is...","{'obj': [{'value': 1348745, 'timestp': '2020-0..."


In [6]:
# Convert some columns from strings to dictionaries
df_cmc_songs.cmc_song_seach = df_cmc_songs.cmc_song_seach.apply(literal_eval)
df_cmc_songs.cmc_track_metadata = df_cmc_songs.cmc_track_metadata.apply(literal_eval)
df_cmc_songs.cmc_track_stats_tiktok = df_cmc_songs.cmc_track_stats_tiktok.apply(literal_eval)

In [7]:
# Check the result
print(f'Column "cmc_song_seach": {type(df_cmc_songs.cmc_song_seach[0])}')
print(f'Column "cmc_track_metadata": {type(df_cmc_songs.cmc_track_metadata[0])}')
print(f'Column "cmc_track_stats_tiktok": {type(df_cmc_songs.cmc_track_stats_tiktok[0])}')

Column "cmc_song_seach": <class 'dict'>
Column "cmc_track_metadata": <class 'dict'>
Column "cmc_track_stats_tiktok": <class 'dict'>


## Data Manipulation

This section is necessary so the desired data can be added to the dataframes

### Get desired data from the Chartmetric dataset

In [8]:
# Add Chartmetric information to the dataframe

# Duration ms
df_cmc_songs['cmc_track_duration_ms'] = [each_row['obj']['duration_ms'] 
                                         if type(each_row) == dict 
                                         else 'not-found' 
                                         for each_row in df_cmc_songs.cmc_track_metadata]

# Tags
df_cmc_songs['cmc_track_tags'] = [each_row['obj']['tags'].lower().split(',')
                                  if (type(each_row) == dict) & (type(each_row['obj']['tags']) == str) 
                                  else 'not-found' 
                                  for each_row in df_cmc_songs.cmc_track_metadata]

# Release date
df_cmc_songs['cmc_track_release_date'] = [each_row['obj']['release_date'] 
                                          if type(each_row) == dict 
                                          else 'not-found' 
                                          for each_row in df_cmc_songs.cmc_track_metadata]

# Chartmetric features
## Key
df_cmc_songs['cmc_track_feat_key'] = [each_row['obj']['cm_audio_features']['key'] 
                                      if type(each_row['obj']['cm_audio_features']) == dict 
                                      else 'not-value' 
                                      for each_row in df_cmc_songs.cmc_track_metadata]

## Mode
df_cmc_songs['cmc_track_feat_mode'] = [each_row['obj']['cm_audio_features']['mode'] 
                                       if type(each_row['obj']['cm_audio_features']) == dict 
                                       else 'not-value' 
                                       for each_row in df_cmc_songs.cmc_track_metadata]

## Danceability %
df_cmc_songs['cmc_track_feat_danceability'] = [each_row['obj']['cm_audio_features']['danceability'] 
                                               if type(each_row['obj']['cm_audio_features']) == dict 
                                               else 'not-value' 
                                               for each_row in df_cmc_songs.cmc_track_metadata]

## Energy %
df_cmc_songs['cmc_track_feat_energy'] = [each_row['obj']['cm_audio_features']['energy'] 
                                         if type(each_row['obj']['cm_audio_features']) == dict 
                                         else 'not-value' 
                                         for each_row in df_cmc_songs.cmc_track_metadata]

## Speechiness
df_cmc_songs['cmc_track_feat_speechiness'] = [each_row['obj']['cm_audio_features']['speechiness'] 
                                              if type(each_row['obj']['cm_audio_features']) == dict 
                                              else 'not-value' 
                                              for each_row in df_cmc_songs.cmc_track_metadata]

## Acousticness %
df_cmc_songs['cmc_track_feat_acousticness'] = [each_row['obj']['cm_audio_features']['acousticness'] 
                                               if type(each_row['obj']['cm_audio_features']) == dict 
                                               else 'not-value' 
                                               for each_row in df_cmc_songs.cmc_track_metadata]

## Instrumentalness
df_cmc_songs['cmc_track_feat_instrumentalness'] = [each_row['obj']['cm_audio_features']['instrumentalness'] 
                                                   if type(each_row['obj']['cm_audio_features']) == dict 
                                                   else 'not-value' 
                                                   for each_row in df_cmc_songs.cmc_track_metadata]

## Liveness %
df_cmc_songs['cmc_track_feat_liveness'] = [each_row['obj']['cm_audio_features']['liveness'] 
                                           if type(each_row['obj']['cm_audio_features']) == dict 
                                           else 'not-value' 
                                           for each_row in df_cmc_songs.cmc_track_metadata]

## Valence %
df_cmc_songs['cmc_track_feat_valence'] = [each_row['obj']['cm_audio_features']['valence'] 
                                          if type(each_row['obj']['cm_audio_features']) == dict 
                                          else 'not-value' 
                                          for each_row in df_cmc_songs.cmc_track_metadata]

## Tempo
df_cmc_songs['cmc_track_feat_tempo'] = [each_row['obj']['cm_audio_features']['tempo'] 
                                        if type(each_row['obj']['cm_audio_features']) == dict 
                                        else 'not-value' 
                                        for each_row in df_cmc_songs.cmc_track_metadata]

## Loudness
df_cmc_songs['cmc_track_feat_loudness'] = [each_row['obj']['cm_audio_features']['loudness'] 
                                           if type(each_row['obj']['cm_audio_features']) == dict 
                                           else 'not-value' 
                                           for each_row in df_cmc_songs.cmc_track_metadata]

# Chartmetric statistics
## Spotify Popularity
df_cmc_songs['cmc_track_stat_sp_pop'] = [each_row['obj']['cm_statistics']['sp_popularity'] 
                                         if type(each_row['obj']['cm_audio_features']) == dict 
                                         else 'not-value' 
                                         for each_row in df_cmc_songs.cmc_track_metadata]

## Number of TikTok posts
df_cmc_songs['cmc_track_stat_tiktok_counts'] = [each_row['obj']['cm_statistics']['tiktok_counts'] 
                                                if type(each_row['obj']['cm_audio_features']) == dict 
                                                else 'not-value' 
                                                for each_row in df_cmc_songs.cmc_track_metadata]

## Youtube views
df_cmc_songs['cmc_track_stat_youtube_views'] = [each_row['obj']['cm_statistics']['youtube_views'] 
                                                if type(each_row['obj']['cm_audio_features']) == dict 
                                                else 'not-value' 
                                                for each_row in df_cmc_songs.cmc_track_metadata]

# TikTok Stats
df_cmc_songs['cmc_tiktok_stats'] = [row['obj'] for row in df_cmc_songs.cmc_track_stats_tiktok]

In [9]:
# Check the result
df_cmc_songs.head()

,song,artists,artists_list,number_artists,sp_id,cmc_song_seach,cmc_id,cmc_track_metadata,cmc_track_stats_tiktok,cmc_track_duration_ms,...,cmc_track_feat_acousticness,cmc_track_feat_instrumentalness,cmc_track_feat_liveness,cmc_track_feat_valence,cmc_track_feat_tempo,cmc_track_feat_loudness,cmc_track_stat_sp_pop,cmc_track_stat_tiktok_counts,cmc_track_stat_youtube_views,cmc_tiktok_stats
0,Roxanne,Arizona Zervas,['Arizona Zervas'],1,696DnlkuDOXcMAnKlTgXXK,"{'obj': {'tracks': [{'id': 27228348, 'name': '...",27228348,"{'obj': {'id': 27228348, 'name': 'ROXANNE', 'i...","{'obj': [{'value': 2100000, 'timestp': '2020-0...",163636,...,0.0522,0,0.46,0.457,116.735,-5.616,88,2300000,96679674,"[{'value': 2100000, 'timestp': '2020-01-19T00:..."
1,Say So,Doja Cat,['Doja Cat'],1,3Dv1eDb0MEgF93GpLXlucZ,"{'obj': {'tracks': [{'id': 26951096, 'name': '...",26951096,"{'obj': {'id': 26951096, 'name': 'Say So', 'is...","{'obj': [{'value': 5300000, 'timestp': '2020-0...",237893,...,0.256,3.57e-06,0.0904,0.786,110.962,-4.577,89,19300000,198853609,"[{'value': 5300000, 'timestp': '2020-01-01T00:..."
2,My Oh My,Camila Cabello feat. DaBaby,"['Camila Cabello', 'DaBaby']",2,3yOlyBJuViE2YSGn3nVE1K,"{'obj': {'tracks': [{'id': 27597895, 'name': '...",27597895,"{'obj': {'id': 27597895, 'name': 'My Oh My', '...","{'obj': [{'value': 791700, 'timestp': '2020-01...",170746,...,0.018,1.29e-05,0.0887,0.383,105.046,-6.024,None,1900000,49709340,"[{'value': 791700, 'timestp': '2020-01-01T00:0..."
3,Moon,Kid Francescoli,['Kid Francescoli'],1,24upABZ8A0sAepfu91sEYr,"{'obj': {'tracks': [{'id': 12263271, 'name': '...",12263271,"{'obj': {'id': 12263271, 'name': 'Moon', 'isrc...","{'obj': [{'value': 382197, 'timestp': '2020-02...",390638,...,0.291,0.863,0.101,0.0578,117.984,-10.002,20,759300,None,"[{'value': 382197, 'timestp': '2020-02-24T00:0..."
4,Vibe,Cookiee Kawaii,['Cookiee Kawaii'],1,4gOgQTv9RYYFZ1uQNnlk3q,"{'obj': {'tracks': [{'id': 25138356, 'name': '...",25138356,"{'obj': {'id': 25138356, 'name': 'Vibe', 'isrc...","{'obj': [{'value': 948913, 'timestp': '2020-03...",84008,...,0.0676,0.00951,0.117,0.169,159.995,-8.74,21,1900000,None,"[{'value': 948913, 'timestp': '2020-03-01T00:0..."


In [10]:
# Remove the unnecessary columns
df_cmc_songs = df_cmc_songs.drop(columns=['sp_id', 'artists', 'artists_list', 'number_artists', 'cmc_id', 
                                          'cmc_song_seach', 'cmc_track_metadata', 'cmc_track_stats_tiktok'])

# Check the result
df_cmc_songs.head()

,song,cmc_track_duration_ms,cmc_track_tags,cmc_track_release_date,cmc_track_feat_key,cmc_track_feat_mode,cmc_track_feat_danceability,cmc_track_feat_energy,cmc_track_feat_speechiness,cmc_track_feat_acousticness,cmc_track_feat_instrumentalness,cmc_track_feat_liveness,cmc_track_feat_valence,cmc_track_feat_tempo,cmc_track_feat_loudness,cmc_track_stat_sp_pop,cmc_track_stat_tiktok_counts,cmc_track_stat_youtube_views,cmc_tiktok_stats
0,Roxanne,163636,"[hip-hop/rap, music, pop, música, pop, musik]",2019-10-10T00:00:00.000Z,6,0,0.621,0.601,0.148,0.0522,0,0.46,0.457,116.735,-5.616,88,2300000,96679674,"[{'value': 2100000, 'timestp': '2020-01-19T00:..."
1,Say So,237893,"[dance, musica, r&b/soul, music]",2019-11-07T00:00:00.000Z,11,0,0.787,0.673,0.158,0.256,3.57e-06,0.0904,0.786,110.962,-4.577,89,19300000,198853609,"[{'value': 5300000, 'timestp': '2020-01-01T00:..."
2,My Oh My,170746,"[pop, music]",2019-12-06T00:00:00.000Z,8,1,0.724,0.491,0.0296,0.018,1.29e-05,0.0887,0.383,105.046,-6.024,None,1900000,49709340,"[{'value': 791700, 'timestp': '2020-01-01T00:0..."
3,Moon,390638,"[pop, music]",2017-03-03T00:00:00.000Z,7,1,0.662,0.657,0.0345,0.291,0.863,0.101,0.0578,117.984,-10.002,20,759300,None,"[{'value': 382197, 'timestp': '2020-02-24T00:0..."
4,Vibe,84008,"[dance, music, r&b/soul]",2019-03-29T00:00:00.000Z,10,1,0.759,0.561,0.384,0.0676,0.00951,0.117,0.169,159.995,-8.74,21,1900000,None,"[{'value': 948913, 'timestp': '2020-03-01T00:0..."


### Merge the both dataframes

In [11]:
# Merge the dataframes
df_songs = pd.merge(left=df_sp_songs, right=df_cmc_songs, on='song')

# Check the result
df_songs

,song,artists,artists_list,number_artists,sp_name,sp_duration_ms,sp_popularity,sp_release_date,sp_explicit,sp_artists_name,...,cmc_track_feat_acousticness,cmc_track_feat_instrumentalness,cmc_track_feat_liveness,cmc_track_feat_valence,cmc_track_feat_tempo,cmc_track_feat_loudness,cmc_track_stat_sp_pop,cmc_track_stat_tiktok_counts,cmc_track_stat_youtube_views,cmc_tiktok_stats
0,Roxanne,Arizona Zervas,[Arizona Zervas],1,ROXANNE,163636,88,2019-10-10,True,[Arizona Zervas],...,0.0522,0,0.46,0.457,116.735,-5.616,88,2300000,96679674,"[{'value': 2100000, 'timestp': '2020-01-19T00:..."
1,Say So,Doja Cat,[Doja Cat],1,Say So,237893,88,2019-11-07,True,[Doja Cat],...,0.256,3.57e-06,0.0904,0.786,110.962,-4.577,89,19300000,198853609,"[{'value': 5300000, 'timestp': '2020-01-01T00:..."
2,My Oh My,Camila Cabello feat. DaBaby,"[Camila Cabello, DaBaby]",2,My Oh My (feat. DaBaby),170746,82,2019-12-06,False,"[Camila Cabello, DaBaby]",...,0.018,1.29e-05,0.0887,0.383,105.046,-6.024,None,1900000,49709340,"[{'value': 791700, 'timestp': '2020-01-01T00:0..."
3,Moon,Kid Francescoli,[Kid Francescoli],1,Moon (And It Went Like),390638,70,2017-03-03,False,[Kid Francescoli],...,0.291,0.863,0.101,0.0578,117.984,-10.002,20,759300,None,"[{'value': 382197, 'timestp': '2020-02-24T00:0..."
4,Vibe,Cookiee Kawaii,[Cookiee Kawaii],1,Vibe (If I Back It Up),83940,72,2019-03-29,True,[Cookiee Kawaii],...,0.0676,0.00951,0.117,0.169,159.995,-8.74,21,1900000,None,"[{'value': 948913, 'timestp': '2020-03-01T00:0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,What the Hell,Avril Lavigne,[Avril Lavigne],1,What the Hell,220706,74,2011-03-08,False,[Avril Lavigne],...,0.00483,0.00956,0.163,0.885,150.081,-5.162,74,1600000,337047363,"[{'value': 315800, 'timestp': '2020-02-14T00:0..."
62,Towards the Sun,Rihanna,[Rihanna],1,"Towards The Sun - From The ""Home"" Soundtrack",273293,55,2015-03-23,False,[Rihanna],...,0.0559,0,0.156,0.241,170.211,-6.143,59,None,912388,"[{'value': 0, 'timestp': '2020-01-07T00:00:00...."
63,I Think I'm OKAY,"Machine Gun Kelly, YUNGBLUD, and Travis Barker","[Machine Gun Kelly, YUNGBLUD, Travis Barker]",3,I Think I'm OKAY (with YUNGBLUD & Travis Barker),169397,81,2019-07-05,True,"[Machine Gun Kelly, YUNGBLUD, Travis Barker]",...,0.0327,0,0.3,0.24,119.94,-4.691,20,254900,50808170,"[{'value': 60100, 'timestp': '2020-01-01T00:00..."
64,Myself,Bazzi,[Bazzi],1,Myself,167552,76,2018-04-12,False,[Bazzi],...,0.465,1.12e-06,0.0338,0.902,195.918,-5.513,77,3700000,43521205,"[{'value': 1348745, 'timestp': '2020-02-24T00:..."


In [12]:
# Check the columns of the new dataframe
df_songs.columns

Index(['song', 'artists', 'artists_list', 'number_artists', 'sp_name',
       'sp_duration_ms', 'sp_popularity', 'sp_release_date', 'sp_explicit',
       'sp_artists_name', 'sp_danceability', 'sp_energy', 'sp_key',
       'sp_loudness', 'sp_mode', 'sp_speechiness', 'sp_acousticness',
       'sp_instrumentalness', 'sp_liveness', 'sp_valence', 'sp_tempo',
       'sp_time_signature', 'cmc_track_duration_ms', 'cmc_track_tags',
       'cmc_track_release_date', 'cmc_track_feat_key', 'cmc_track_feat_mode',
       'cmc_track_feat_danceability', 'cmc_track_feat_energy',
       'cmc_track_feat_speechiness', 'cmc_track_feat_acousticness',
       'cmc_track_feat_instrumentalness', 'cmc_track_feat_liveness',
       'cmc_track_feat_valence', 'cmc_track_feat_tempo',
       'cmc_track_feat_loudness', 'cmc_track_stat_sp_pop',
       'cmc_track_stat_tiktok_counts', 'cmc_track_stat_youtube_views',
       'cmc_tiktok_stats'],
      dtype='object')

In [13]:
# Add a song id
df_songs['song_id'] = [index + 1 for index in df_songs.index]

# Reorder columns
cols = list(df_songs.columns)
cols = [cols[-1]] + cols[:-1]
df_songs = df_songs[cols]

# Check the result
df_songs

,song_id,song,artists,artists_list,number_artists,sp_name,sp_duration_ms,sp_popularity,sp_release_date,sp_explicit,...,cmc_track_feat_acousticness,cmc_track_feat_instrumentalness,cmc_track_feat_liveness,cmc_track_feat_valence,cmc_track_feat_tempo,cmc_track_feat_loudness,cmc_track_stat_sp_pop,cmc_track_stat_tiktok_counts,cmc_track_stat_youtube_views,cmc_tiktok_stats
0,1,Roxanne,Arizona Zervas,[Arizona Zervas],1,ROXANNE,163636,88,2019-10-10,True,...,0.0522,0,0.46,0.457,116.735,-5.616,88,2300000,96679674,"[{'value': 2100000, 'timestp': '2020-01-19T00:..."
1,2,Say So,Doja Cat,[Doja Cat],1,Say So,237893,88,2019-11-07,True,...,0.256,3.57e-06,0.0904,0.786,110.962,-4.577,89,19300000,198853609,"[{'value': 5300000, 'timestp': '2020-01-01T00:..."
2,3,My Oh My,Camila Cabello feat. DaBaby,"[Camila Cabello, DaBaby]",2,My Oh My (feat. DaBaby),170746,82,2019-12-06,False,...,0.018,1.29e-05,0.0887,0.383,105.046,-6.024,None,1900000,49709340,"[{'value': 791700, 'timestp': '2020-01-01T00:0..."
3,4,Moon,Kid Francescoli,[Kid Francescoli],1,Moon (And It Went Like),390638,70,2017-03-03,False,...,0.291,0.863,0.101,0.0578,117.984,-10.002,20,759300,None,"[{'value': 382197, 'timestp': '2020-02-24T00:0..."
4,5,Vibe,Cookiee Kawaii,[Cookiee Kawaii],1,Vibe (If I Back It Up),83940,72,2019-03-29,True,...,0.0676,0.00951,0.117,0.169,159.995,-8.74,21,1900000,None,"[{'value': 948913, 'timestp': '2020-03-01T00:0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,62,What the Hell,Avril Lavigne,[Avril Lavigne],1,What the Hell,220706,74,2011-03-08,False,...,0.00483,0.00956,0.163,0.885,150.081,-5.162,74,1600000,337047363,"[{'value': 315800, 'timestp': '2020-02-14T00:0..."
62,63,Towards the Sun,Rihanna,[Rihanna],1,"Towards The Sun - From The ""Home"" Soundtrack",273293,55,2015-03-23,False,...,0.0559,0,0.156,0.241,170.211,-6.143,59,None,912388,"[{'value': 0, 'timestp': '2020-01-07T00:00:00...."
63,64,I Think I'm OKAY,"Machine Gun Kelly, YUNGBLUD, and Travis Barker","[Machine Gun Kelly, YUNGBLUD, Travis Barker]",3,I Think I'm OKAY (with YUNGBLUD & Travis Barker),169397,81,2019-07-05,True,...,0.0327,0,0.3,0.24,119.94,-4.691,20,254900,50808170,"[{'value': 60100, 'timestp': '2020-01-01T00:00..."
64,65,Myself,Bazzi,[Bazzi],1,Myself,167552,76,2018-04-12,False,...,0.465,1.12e-06,0.0338,0.902,195.918,-5.513,77,3700000,43521205,"[{'value': 1348745, 'timestp': '2020-02-24T00:..."


### Create a dataframe for music genre

In [14]:
# Create a new dataframe
df_songs_genres = df_songs.loc[:, ['song_id', 'song', 'sp_name', 'cmc_track_tags']]

# Rename some columns
df_songs_genres.rename(columns={'cmc_track_tags': 'genre'}, inplace=True)

# Check the result
df_songs_genres

,song_id,song,sp_name,genre
0,1,Roxanne,ROXANNE,"[hip-hop/rap, music, pop, música, pop, musik]"
1,2,Say So,Say So,"[dance, musica, r&b/soul, music]"
2,3,My Oh My,My Oh My (feat. DaBaby),"[pop, music]"
3,4,Moon,Moon (And It Went Like),"[pop, music]"
4,5,Vibe,Vibe (If I Back It Up),"[dance, music, r&b/soul]"
...,...,...,...,...
61,62,What the Hell,What the Hell,"[ポップ, pop]"
62,63,Towards the Sun,"Towards The Sun - From The ""Home"" Soundtrack",[soundtrack]
63,64,I Think I'm OKAY,I Think I'm OKAY (with YUNGBLUD & Travis Barker),"[alternative, music, hip-hop/rap, music]"
64,65,Myself,Myself,"[pop, music]"


In [15]:
# Use explode
df_songs_genres = df_songs_genres.explode('genre')

# Reset the index
df_songs_genres = df_songs_genres.reset_index()

# Check the result
df_songs_genres

,index,song_id,song,sp_name,genre
0,0,1,Roxanne,ROXANNE,hip-hop/rap
1,0,1,Roxanne,ROXANNE,music
2,0,1,Roxanne,ROXANNE,pop
3,0,1,Roxanne,ROXANNE,música
4,0,1,Roxanne,ROXANNE,pop
...,...,...,...,...,...
221,64,65,Myself,Myself,pop
222,64,65,Myself,Myself,music
223,65,66,Stronger,Stronger,hip-hop/rap
224,65,66,Stronger,Stronger,hip-hop/rap


In [16]:
# Get a list of unique genres
list_unique_genres = list(df_songs_genres.genre.unique())

# Check the result
list_unique_genres

['hip-hop/rap',
 'music',
 'pop',
 'música',
 'musik',
 'dance',
 'musica',
 'r&b/soul',
 'house',
 'rap',
 'rock',
 'electronic',
 'alternativo',
 'urbano latino',
 'latino',
 'latin',
 'instrumental',
 'musique',
 'muziek',
 '舞曲',
 '音乐',
 'singer/songwriter',
 'reggae',
 'modern dancehall',
 'christian & gospel',
 'alternativa',
 'alternative',
 'hiphop/rap',
 '嘻哈/说唱',
 'electrónica',
 'électronique',
 'fitness & workout',
 'танцевальная музыка',
 'dirty south',
 'contemporary r&b',
 'not-found',
 'west coast rap',
 'ポップ',
 'soundtrack']

Among the genres, there some items listed, such as "music" and "singer/songwriter", which are not a genre. So the rows containing them will be removed.

In [17]:
# Filter the dataset
df_songs_genres = df_songs_genres[~((df_songs_genres.genre == 'music') | 
                                    (df_songs_genres.genre == 'musique') |
                                    (df_songs_genres.genre == 'música') |
                                    (df_songs_genres.genre == 'musik') |
                                    (df_songs_genres.genre == 'musica') |
                                    (df_songs_genres.genre == 'muziek') |
                                    (df_songs_genres.genre == '音乐') |
                                    (df_songs_genres.genre == 'singer/songwriter') |
                                    (df_songs_genres.genre == 'soundtrack') |
                                    (df_songs_genres.genre == 'fitness & workout') |
                                    (df_songs_genres.genre == 'not-found'))].reset_index(drop=True)

# Check the result
df_songs_genres

,index,song_id,song,sp_name,genre
0,0,1,Roxanne,ROXANNE,hip-hop/rap
1,0,1,Roxanne,ROXANNE,pop
2,0,1,Roxanne,ROXANNE,pop
3,1,2,Say So,Say So,dance
4,1,2,Say So,Say So,r&b/soul
...,...,...,...,...,...
134,63,64,I Think I'm OKAY,I Think I'm OKAY (with YUNGBLUD & Travis Barker),alternative
135,63,64,I Think I'm OKAY,I Think I'm OKAY (with YUNGBLUD & Travis Barker),hip-hop/rap
136,64,65,Myself,Myself,pop
137,65,66,Stronger,Stronger,hip-hop/rap


In [18]:
# Check unique genres
list_unique_genres = list(df_songs_genres.genre.unique())

# Check the result
list_unique_genres

['hip-hop/rap',
 'pop',
 'dance',
 'r&b/soul',
 'house',
 'rap',
 'rock',
 'electronic',
 'alternativo',
 'urbano latino',
 'latino',
 'latin',
 'instrumental',
 '舞曲',
 'reggae',
 'modern dancehall',
 'christian & gospel',
 'alternativa',
 'alternative',
 'hiphop/rap',
 '嘻哈/说唱',
 'electrónica',
 'électronique',
 'танцевальная музыка',
 'dirty south',
 'contemporary r&b',
 'west coast rap',
 'ポップ']

There are some genres that are listed in different languages or are written differently. So, we will standardize them.

In [19]:
# Hip-hop/Rap
df_songs_genres.replace(to_replace=['hiphop/rap', '嘻哈/说唱'], value='hip-hop/rap', regex=True, inplace=True)

# Dance
df_songs_genres.replace(to_replace=['танцевальная музыка', '舞曲'], value='dance', regex=True, inplace=True)

# Pop
df_songs_genres.replace(to_replace='ポップ', value='pop', regex=True, inplace=True)

# Eletronic
df_songs_genres.replace(to_replace=['electrónica', 'électronique'], value='electronic', regex=True, inplace=True)

# Alternative
df_songs_genres.replace(to_replace=['alternativo', 'alternativa', 'alternative'], 
                        value='alternative', regex=True, inplace=True)

# latin
df_songs_genres.replace(to_replace=['urbano latino', 'latino'], value='latin', regex=True, inplace=True)

In [20]:
# Check unique genres
list_unique_genres = list(df_songs_genres.genre.unique())

# Check the result
list_unique_genres

['hip-hop/rap',
 'pop',
 'dance',
 'r&b/soul',
 'house',
 'rap',
 'rock',
 'electronic',
 'alternative',
 'latin',
 'instrumental',
 'reggae',
 'modern dancehall',
 'christian & gospel',
 'dirty south',
 'contemporary r&b',
 'west coast rap']

### Create a dataframe for audio features

In [21]:
# Calculate the mean
instrumentalness_mean = df_songs.sp_instrumentalness.mean()
speechiness_mean = df_songs.sp_speechiness.mean()
acousticness_mean = df_songs.sp_acousticness.mean()
liveness_mean = df_songs.sp_liveness.mean()
danceability_mean = df_songs.sp_danceability.mean()
energy_mean = df_songs.sp_energy.mean()
valence_mean = df_songs.sp_valence.mean()



In [22]:
# Calculate the standard deviation
instrumentalness_std = df_songs.sp_instrumentalness.std()
speechiness_std = df_songs.sp_speechiness.std()
acousticness_std = df_songs.sp_acousticness.std()
liveness_std = df_songs.sp_liveness.std()
danceability_std = df_songs.sp_danceability.std()
energy_std = df_songs.sp_energy.std()
valence_std = df_songs.sp_valence.std()

In [23]:
# Create lists
list_audio_features = ['instrumentalness', 'speechiness', 'acousticness', 'liveness', 'danceability', 'energy', 'valence']

list_audio_features_mean = [instrumentalness_mean, speechiness_mean, acousticness_mean, liveness_mean, danceability_mean,
                            energy_mean, valence_mean]

list_audio_features_std = [instrumentalness_std, speechiness_std, acousticness_std, liveness_std, danceability_std,
                           energy_std, valence_std]

In [24]:
# Create the new dataframe
df_songs_audio_features_summary = pd.DataFrame(zip(list_audio_features, 
                                                   list_audio_features_mean, 
                                                   list_audio_features_std), columns=['audio_feature', 'mean', 'std'])

# Check the result
df_songs_audio_features_summary

,audio_feature,mean,std
0,instrumentalness,0.024681,0.118616
1,speechiness,0.155221,0.147650
2,acousticness,0.230803,0.266136
3,liveness,0.192956,0.154887
4,danceability,0.717515,0.160793
5,energy,0.627455,0.184296
6,valence,0.558127,0.218826


### Create a dataframe for artists

In [25]:
# Create a new dataframe
df_songs_artists = df_songs.loc[:, ['song_id', 'song', 'sp_name', 'artists', 'artists_list', 'number_artists', 
                                    'sp_artists_name']]

# Check the result
df_songs_artists

,song_id,song,sp_name,artists,artists_list,number_artists,sp_artists_name
0,1,Roxanne,ROXANNE,Arizona Zervas,[Arizona Zervas],1,[Arizona Zervas]
1,2,Say So,Say So,Doja Cat,[Doja Cat],1,[Doja Cat]
2,3,My Oh My,My Oh My (feat. DaBaby),Camila Cabello feat. DaBaby,"[Camila Cabello, DaBaby]",2,"[Camila Cabello, DaBaby]"
3,4,Moon,Moon (And It Went Like),Kid Francescoli,[Kid Francescoli],1,[Kid Francescoli]
4,5,Vibe,Vibe (If I Back It Up),Cookiee Kawaii,[Cookiee Kawaii],1,[Cookiee Kawaii]
...,...,...,...,...,...,...,...
61,62,What the Hell,What the Hell,Avril Lavigne,[Avril Lavigne],1,[Avril Lavigne]
62,63,Towards the Sun,"Towards The Sun - From The ""Home"" Soundtrack",Rihanna,[Rihanna],1,[Rihanna]
63,64,I Think I'm OKAY,I Think I'm OKAY (with YUNGBLUD & Travis Barker),"Machine Gun Kelly, YUNGBLUD, and Travis Barker","[Machine Gun Kelly, YUNGBLUD, Travis Barker]",3,"[Machine Gun Kelly, YUNGBLUD, Travis Barker]"
64,65,Myself,Myself,Bazzi,[Bazzi],1,[Bazzi]


In [26]:
# Check if the songs has a feat. artist
df_songs_artists['feat'] = ['Yes' if 'eat.' in artists else 'No' for artists in df_songs_artists.artists]
df_songs_artists['feat_artist'] = [df_songs_artists.iloc[index, 4][-1] 
                                   if df_songs_artists.iloc[index, 6] == 'Yes' 
                                   else 'no-feat' 
                                   for index in df_songs_artists.index]

# Check result
df_songs_artists

,song_id,song,sp_name,artists,artists_list,number_artists,sp_artists_name,feat,feat_artist
0,1,Roxanne,ROXANNE,Arizona Zervas,[Arizona Zervas],1,[Arizona Zervas],No,no-feat
1,2,Say So,Say So,Doja Cat,[Doja Cat],1,[Doja Cat],No,no-feat
2,3,My Oh My,My Oh My (feat. DaBaby),Camila Cabello feat. DaBaby,"[Camila Cabello, DaBaby]",2,"[Camila Cabello, DaBaby]",Yes,no-feat
3,4,Moon,Moon (And It Went Like),Kid Francescoli,[Kid Francescoli],1,[Kid Francescoli],No,no-feat
4,5,Vibe,Vibe (If I Back It Up),Cookiee Kawaii,[Cookiee Kawaii],1,[Cookiee Kawaii],No,no-feat
...,...,...,...,...,...,...,...,...,...
61,62,What the Hell,What the Hell,Avril Lavigne,[Avril Lavigne],1,[Avril Lavigne],No,no-feat
62,63,Towards the Sun,"Towards The Sun - From The ""Home"" Soundtrack",Rihanna,[Rihanna],1,[Rihanna],No,no-feat
63,64,I Think I'm OKAY,I Think I'm OKAY (with YUNGBLUD & Travis Barker),"Machine Gun Kelly, YUNGBLUD, and Travis Barker","[Machine Gun Kelly, YUNGBLUD, Travis Barker]",3,"[Machine Gun Kelly, YUNGBLUD, Travis Barker]",No,no-feat
64,65,Myself,Myself,Bazzi,[Bazzi],1,[Bazzi],No,no-feat


In [27]:
# Use explode
df_songs_artists = df_songs_artists.explode('sp_artists_name')

# Reset the index
df_songs_artists = df_songs_artists.reset_index(drop=True)

# Check the result
df_songs_artists

,song_id,song,sp_name,artists,artists_list,number_artists,sp_artists_name,feat,feat_artist
0,1,Roxanne,ROXANNE,Arizona Zervas,[Arizona Zervas],1,Arizona Zervas,No,no-feat
1,2,Say So,Say So,Doja Cat,[Doja Cat],1,Doja Cat,No,no-feat
2,3,My Oh My,My Oh My (feat. DaBaby),Camila Cabello feat. DaBaby,"[Camila Cabello, DaBaby]",2,Camila Cabello,Yes,no-feat
3,3,My Oh My,My Oh My (feat. DaBaby),Camila Cabello feat. DaBaby,"[Camila Cabello, DaBaby]",2,DaBaby,Yes,no-feat
4,4,Moon,Moon (And It Went Like),Kid Francescoli,[Kid Francescoli],1,Kid Francescoli,No,no-feat
...,...,...,...,...,...,...,...,...,...
83,64,I Think I'm OKAY,I Think I'm OKAY (with YUNGBLUD & Travis Barker),"Machine Gun Kelly, YUNGBLUD, and Travis Barker","[Machine Gun Kelly, YUNGBLUD, Travis Barker]",3,Machine Gun Kelly,No,no-feat
84,64,I Think I'm OKAY,I Think I'm OKAY (with YUNGBLUD & Travis Barker),"Machine Gun Kelly, YUNGBLUD, and Travis Barker","[Machine Gun Kelly, YUNGBLUD, Travis Barker]",3,YUNGBLUD,No,no-feat
85,64,I Think I'm OKAY,I Think I'm OKAY (with YUNGBLUD & Travis Barker),"Machine Gun Kelly, YUNGBLUD, and Travis Barker","[Machine Gun Kelly, YUNGBLUD, Travis Barker]",3,Travis Barker,No,no-feat
86,65,Myself,Myself,Bazzi,[Bazzi],1,Bazzi,No,no-feat


### Create a dataframe for Chartmetric TikTok stats

In [28]:
# Create a new dataframe
df_songs_tiktok_stats = df_songs.loc[:, ['song_id', 'song', 'sp_name', 'cmc_tiktok_stats']]

# Rename some columns
df_songs_tiktok_stats.rename(columns={'cmc_tiktok_stats': 'tiktok_stats'}, inplace=True)

# Check the result
df_songs_tiktok_stats

,song_id,song,sp_name,tiktok_stats
0,1,Roxanne,ROXANNE,"[{'value': 2100000, 'timestp': '2020-01-19T00:..."
1,2,Say So,Say So,"[{'value': 5300000, 'timestp': '2020-01-01T00:..."
2,3,My Oh My,My Oh My (feat. DaBaby),"[{'value': 791700, 'timestp': '2020-01-01T00:0..."
3,4,Moon,Moon (And It Went Like),"[{'value': 382197, 'timestp': '2020-02-24T00:0..."
4,5,Vibe,Vibe (If I Back It Up),"[{'value': 948913, 'timestp': '2020-03-01T00:0..."
...,...,...,...,...
61,62,What the Hell,What the Hell,"[{'value': 315800, 'timestp': '2020-02-14T00:0..."
62,63,Towards the Sun,"Towards The Sun - From The ""Home"" Soundtrack","[{'value': 0, 'timestp': '2020-01-07T00:00:00...."
63,64,I Think I'm OKAY,I Think I'm OKAY (with YUNGBLUD & Travis Barker),"[{'value': 60100, 'timestp': '2020-01-01T00:00..."
64,65,Myself,Myself,"[{'value': 1348745, 'timestp': '2020-02-24T00:..."


In [29]:
# Use explode
df_songs_tiktok_stats = df_songs_tiktok_stats.explode('tiktok_stats')

# Reset the index
df_songs_tiktok_stats = df_songs_tiktok_stats.reset_index()

# Check the result
df_songs_tiktok_stats

,index,song_id,song,sp_name,tiktok_stats
0,0,1,Roxanne,ROXANNE,"{'value': 2100000, 'timestp': '2020-01-19T00:0..."
1,0,1,Roxanne,ROXANNE,"{'value': 2100000, 'timestp': '2020-01-20T00:0..."
2,0,1,Roxanne,ROXANNE,"{'value': 2100000, 'timestp': '2020-01-21T00:0..."
3,0,1,Roxanne,ROXANNE,"{'value': 2200000, 'timestp': '2020-01-28T00:0..."
4,0,1,Roxanne,ROXANNE,"{'value': 2200000, 'timestp': '2020-01-29T00:0..."
...,...,...,...,...,...
10265,65,66,Stronger,Stronger,"{'value': 16200, 'timestp': '2020-09-13T00:00:..."
10266,65,66,Stronger,Stronger,"{'value': 16200, 'timestp': '2020-09-14T00:00:..."
10267,65,66,Stronger,Stronger,"{'value': 16200, 'timestp': '2020-09-15T00:00:..."
10268,65,66,Stronger,Stronger,"{'value': 16200, 'timestp': '2020-09-16T00:00:..."


In [30]:
# Create new columns with information of stats about TikTok
df_songs_tiktok_stats['value'] = [row['value'] if row == row else row for row in df_songs_tiktok_stats.tiktok_stats]
df_songs_tiktok_stats['timestp'] = [row['timestp'] if row == row else row 
                                    for row in df_songs_tiktok_stats.tiktok_stats]
df_songs_tiktok_stats['diff'] = [row['diff'] if row == row else row for row in df_songs_tiktok_stats.tiktok_stats]

# Check the result
df_songs_tiktok_stats.head()

,index,song_id,song,sp_name,tiktok_stats,value,timestp,diff
0,0,1,Roxanne,ROXANNE,"{'value': 2100000, 'timestp': '2020-01-19T00:0...",2100000.0,2020-01-19T00:00:00.000Z,NaN
1,0,1,Roxanne,ROXANNE,"{'value': 2100000, 'timestp': '2020-01-20T00:0...",2100000.0,2020-01-20T00:00:00.000Z,0.0
2,0,1,Roxanne,ROXANNE,"{'value': 2100000, 'timestp': '2020-01-21T00:0...",2100000.0,2020-01-21T00:00:00.000Z,0.0
3,0,1,Roxanne,ROXANNE,"{'value': 2200000, 'timestp': '2020-01-28T00:0...",2200000.0,2020-01-28T00:00:00.000Z,100000.0
4,0,1,Roxanne,ROXANNE,"{'value': 2200000, 'timestp': '2020-01-29T00:0...",2200000.0,2020-01-29T00:00:00.000Z,0.0


# Artists

## Import the dataset

The dataframe with the Chartmetric information has already data from Spotify.

In [31]:
# Import the dataset
df_cmc_artists = pd.read_csv('exported_df/cmc_artists_info.csv')

# Remove unnecessary columns
df_cmc_artists = df_cmc_artists.drop(columns=['sp_id', 'cmc_id'])

# Check the result
df_cmc_artists

,artist,sp_genres,sp_popularity,sp_followers,cmc_search,cmc_art_metadata,cmc_art_sp_city,cmc_art_tiktok_audience
0,Arizona Zervas,"['pop rap', 'rap', 'rhode island rap']",80,486779,"{'obj': {'artists': [{'id': 64150, 'name': 'Ar...","{'obj': {'id': 64150, 'name': 'Arizona Zervas'...",{'obj': {'Atlanta': [{'timestp': '2020-03-21T0...,{'obj': {'top_countries': [{'name': 'United St...
1,Doja Cat,"['la indie', 'pop']",88,3445699,"{'obj': {'artists': [{'id': 217671, 'name': 'D...","{'obj': {'id': 217671, 'name': 'Doja Cat', 'cr...",{'obj': {'Atlanta': [{'timestp': '2020-03-21T0...,{'obj': {'top_countries': [{'name': 'United St...
2,Camila Cabello,"['dance pop', 'pop', 'post-teen pop']",87,17989469,"{'obj': {'artists': [{'id': 454302, 'name': 'C...","{'obj': {'id': 454302, 'name': 'Camila Cabello...",{'obj': {'Brisbane': [{'timestp': '2020-03-21T...,{'obj': {'top_countries': [{'name': 'United St...
3,DaBaby,"['north carolina hip hop', 'rap']",95,4326193,"{'obj': {'artists': [{'id': 398544, 'name': 'D...","{'obj': {'id': 398544, 'name': 'DaBaby', 'crea...",{'obj': {'Atlanta': [{'timestp': '2020-03-21T0...,"{'obj': {'top_countries': [], 'audience_gender..."
4,Kid Francescoli,"['electronica', 'french indie pop', 'french in...",61,82296,"{'obj': {'artists': [{'id': 147365, 'name': 'K...","{'obj': {'id': 147365, 'name': 'Kid Francescol...",{'obj': {'Berlin': [{'timestp': '2020-03-21T00...,"{'obj': {'top_countries': [], 'audience_gender..."
...,...,...,...,...,...,...,...,...
83,Machine Gun Kelly,"['ohio hip hop', 'pop rap', 'rap']",86,2454191,"{'obj': {'artists': [{'id': 3991, 'name': 'Mac...","{'obj': {'id': 3991, 'name': 'Machine Gun Kell...",{'obj': {'Atlanta': [{'timestp': '2020-03-21T0...,{'obj': {'top_countries': [{'name': 'United St...
84,YUNGBLUD,"['british indie rock', 'modern alternative roc...",78,1085134,"{'obj': {'artists': [{'id': 558951, 'name': 'Y...","{'obj': {'id': 558951, 'name': 'YUNGBLUD', 'cr...",{'obj': {'Atlanta': [{'timestp': '2020-03-21T0...,{'obj': {'top_countries': [{'name': 'United St...
85,Travis Barker,['rap rock'],77,252326,"{'obj': {'artists': [{'id': 216839, 'name': 'T...","{'obj': {'id': 216839, 'name': 'Travis Barker'...",{'obj': {'Atlanta': [{'timestp': '2020-03-21T0...,"{'obj': {'top_countries': [], 'audience_gender..."
86,Bazzi,"['pop', 'post-teen pop']",83,3671755,"{'obj': {'artists': [{'id': 213807, 'name': 'B...","{'obj': {'id': 213807, 'name': 'Bazzi', 'creat...",{'obj': {'Atlanta': [{'timestp': '2020-03-21T0...,{'obj': {'top_countries': [{'name': 'United St...


In [32]:
# Convert some columns from strings to dictionaries
df_cmc_artists.sp_genres = df_cmc_artists.sp_genres.apply(literal_eval)
df_cmc_artists.cmc_search = df_cmc_artists.cmc_search.apply(literal_eval)
df_cmc_artists.cmc_art_metadata = df_cmc_artists.cmc_art_metadata.apply(literal_eval)
df_cmc_artists.cmc_art_sp_city = df_cmc_artists.cmc_art_sp_city.apply(literal_eval)
df_cmc_artists.cmc_art_tiktok_audience = df_cmc_artists.cmc_art_tiktok_audience.apply(literal_eval)

In [33]:
# Check the result
print(f'Column "sp_genres": {type(df_cmc_artists.sp_genres[0])}')
print(f'Column "cmc_search": {type(df_cmc_artists.cmc_search[0])}')
print(f'Column "cmc_art_metadata": {type(df_cmc_artists.cmc_art_metadata[0])}')
print(f'Column "cmc_art_sp_city": {type(df_cmc_artists.cmc_art_sp_city[0])}')
print(f'Column "cmc_art_tiktok_audience": {type(df_cmc_artists.cmc_art_tiktok_audience[0])}')

Column "sp_genres": <class 'list'>
Column "cmc_search": <class 'dict'>
Column "cmc_art_metadata": <class 'dict'>
Column "cmc_art_sp_city": <class 'dict'>
Column "cmc_art_tiktok_audience": <class 'dict'>


## Data Manipulation

This section is necessary so the desired data can be added to the dataframes

### Get desired data from the Chartmetric dataset

In [34]:
# Add Chartmetric information to the dataframe
df_cmc_artists['cmc_code2'] = [row['obj']['artists'][0]['code2'] for row in df_cmc_artists.cmc_search]
df_cmc_artists['cmc_tags'] = [row['obj']['artists'][0]['tags'] for row in df_cmc_artists.cmc_search]
df_cmc_artists['cmc_sp_monthly_listeners'] = [row['obj']['artists'][0]['sp_monthly_listeners'] 
                                              for row in df_cmc_artists.cmc_search]
df_cmc_artists['cm_artist_rank'] = [row['obj']['artists'][0]['cm_artist_rank'] for row in df_cmc_artists.cmc_search]
df_cmc_artists['sp_popularity_rank'] = [row['obj']['cm_statistics']['sp_popularity_rank'] 
                                        for row in df_cmc_artists.cmc_art_metadata]
df_cmc_artists['sp_popularity_rank'] = [row['obj']['cm_statistics']['sp_popularity_rank'] 
                                        for row in df_cmc_artists.cmc_art_metadata]
df_cmc_artists['sp_monthly_listeners_rank'] = [row['obj']['cm_statistics']['sp_monthly_listeners_rank'] 
                                               for row in df_cmc_artists.cmc_art_metadata]
df_cmc_artists['youtube_daily_video_views_rank'] = [row['obj']['cm_statistics']['youtube_daily_video_views_rank'] 
                                                    for row in df_cmc_artists.cmc_art_metadata]
df_cmc_artists['youtube_monthly_video_views_rank'] = [row['obj']['cm_statistics']['youtube_monthly_video_views_rank'] 
                                                      for row in df_cmc_artists.cmc_art_metadata]
df_cmc_artists['sp_where_people_listen'] = [row['obj']['cm_statistics']['sp_where_people_listen'] 
                                            for row in df_cmc_artists.cmc_art_metadata]

In [35]:
# Remove unnecessary columns
df_cmc_artists = df_cmc_artists.drop(columns=['cmc_search', 'cmc_art_metadata'])

In [36]:
# Check the result
df_cmc_artists.head()

,artist,sp_genres,sp_popularity,sp_followers,cmc_art_sp_city,cmc_art_tiktok_audience,cmc_code2,cmc_tags,cmc_sp_monthly_listeners,cm_artist_rank,sp_popularity_rank,sp_monthly_listeners_rank,youtube_daily_video_views_rank,youtube_monthly_video_views_rank,sp_where_people_listen
0,Arizona Zervas,"[pop rap, rap, rhode island rap]",80,486779,{'obj': {'Atlanta': [{'timestp': '2020-03-21T0...,{'obj': {'top_countries': [{'name': 'United St...,us,"[pop, pop rap]",15313268,353,459,215,660,562,"[{'name': 'Chicago', 'code2': 'us', 'listeners..."
1,Doja Cat,"[la indie, pop]",88,3445699,{'obj': {'Atlanta': [{'timestp': '2020-03-21T0...,{'obj': {'top_countries': [{'name': 'United St...,us,"[pop, pop rap, la indie, electropop]",31770199,61,101,54,94,100,"[{'name': 'Mexico City', 'code2': 'mx', 'liste..."
2,Camila Cabello,"[dance pop, pop, post-teen pop]",87,17989469,{'obj': {'Brisbane': [{'timestp': '2020-03-21T...,{'obj': {'top_countries': [{'name': 'United St...,us,"[pop, dance pop, post-teen pop]",36816546,54,122,33,124,118,"[{'name': 'Mexico City', 'code2': 'mx', 'liste..."
3,DaBaby,"[north carolina hip hop, rap]",95,4326193,{'obj': {'Atlanta': [{'timestp': '2020-03-21T0...,"{'obj': {'top_countries': [], 'audience_gender...",us,"[rap, nc hip hop]",41793864,41,13,26,101,91,"[{'name': 'Chicago', 'code2': 'us', 'listeners..."
4,Kid Francescoli,"[electronica, french indie pop, french indietr...",61,82296,{'obj': {'Berlin': [{'timestp': '2020-03-21T00...,"{'obj': {'top_countries': [], 'audience_gender...",fr,"[french indie pop, french indietronica, electr...",1316092,6839,6189,4546,0,0,"[{'name': 'Paris', 'code2': 'fr', 'listeners':..."


# Final Dataframes

## Songs - General

This dataframe contains all the general information about each song.

In [37]:
# Dataframe
df_songs.head()

,song_id,song,artists,artists_list,number_artists,sp_name,sp_duration_ms,sp_popularity,sp_release_date,sp_explicit,...,cmc_track_feat_acousticness,cmc_track_feat_instrumentalness,cmc_track_feat_liveness,cmc_track_feat_valence,cmc_track_feat_tempo,cmc_track_feat_loudness,cmc_track_stat_sp_pop,cmc_track_stat_tiktok_counts,cmc_track_stat_youtube_views,cmc_tiktok_stats
0,1,Roxanne,Arizona Zervas,[Arizona Zervas],1,ROXANNE,163636,88,2019-10-10,True,...,0.0522,0,0.46,0.457,116.735,-5.616,88,2300000,96679674,"[{'value': 2100000, 'timestp': '2020-01-19T00:..."
1,2,Say So,Doja Cat,[Doja Cat],1,Say So,237893,88,2019-11-07,True,...,0.256,3.57e-06,0.0904,0.786,110.962,-4.577,89,19300000,198853609,"[{'value': 5300000, 'timestp': '2020-01-01T00:..."
2,3,My Oh My,Camila Cabello feat. DaBaby,"[Camila Cabello, DaBaby]",2,My Oh My (feat. DaBaby),170746,82,2019-12-06,False,...,0.018,1.29e-05,0.0887,0.383,105.046,-6.024,None,1900000,49709340,"[{'value': 791700, 'timestp': '2020-01-01T00:0..."
3,4,Moon,Kid Francescoli,[Kid Francescoli],1,Moon (And It Went Like),390638,70,2017-03-03,False,...,0.291,0.863,0.101,0.0578,117.984,-10.002,20,759300,None,"[{'value': 382197, 'timestp': '2020-02-24T00:0..."
4,5,Vibe,Cookiee Kawaii,[Cookiee Kawaii],1,Vibe (If I Back It Up),83940,72,2019-03-29,True,...,0.0676,0.00951,0.117,0.169,159.995,-8.74,21,1900000,None,"[{'value': 948913, 'timestp': '2020-03-01T00:0..."


In [38]:
# Metadata about the first dataframe
df_songs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66 entries, 0 to 65
Data columns (total 41 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   song_id                          66 non-null     int64  
 1   song                             66 non-null     object 
 2   artists                          66 non-null     object 
 3   artists_list                     66 non-null     object 
 4   number_artists                   66 non-null     int64  
 5   sp_name                          66 non-null     object 
 6   sp_duration_ms                   66 non-null     int64  
 7   sp_popularity                    66 non-null     int64  
 8   sp_release_date                  66 non-null     object 
 9   sp_explicit                      66 non-null     bool   
 10  sp_artists_name                  66 non-null     object 
 11  sp_danceability                  66 non-null     float64
 12  sp_energy               

In [39]:
# Export
df_songs.to_csv('data/proj03-df-songs-data-general.csv')

## Songs - Genres

This dataframe contains the music genres of each song.

In [40]:
# Dataframe
df_songs_genres.head()

,index,song_id,song,sp_name,genre
0,0,1,Roxanne,ROXANNE,hip-hop/rap
1,0,1,Roxanne,ROXANNE,pop
2,0,1,Roxanne,ROXANNE,pop
3,1,2,Say So,Say So,dance
4,1,2,Say So,Say So,r&b/soul


In [41]:
# Metadata about the daframe
df_songs_genres.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139 entries, 0 to 138
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   index    139 non-null    int64 
 1   song_id  139 non-null    int64 
 2   song     139 non-null    object
 3   sp_name  139 non-null    object
 4   genre    139 non-null    object
dtypes: int64(2), object(3)
memory usage: 5.6+ KB


In [42]:
# Export
df_songs_genres.to_csv('data/proj03-df-songs-data-genres.csv')

## Songs - Audio Features

In [43]:
# Dataframe
df_songs_audio_features_summary

,audio_feature,mean,std
0,instrumentalness,0.024681,0.118616
1,speechiness,0.155221,0.147650
2,acousticness,0.230803,0.266136
3,liveness,0.192956,0.154887
4,danceability,0.717515,0.160793
5,energy,0.627455,0.184296
6,valence,0.558127,0.218826


In [44]:
# Export
df_songs_audio_features_summary.to_csv('data/proj03-df-songs-data-audio-features.csv')

## Songs - Artists

In [45]:
# Dataframe
df_songs_artists

,song_id,song,sp_name,artists,artists_list,number_artists,sp_artists_name,feat,feat_artist
0,1,Roxanne,ROXANNE,Arizona Zervas,[Arizona Zervas],1,Arizona Zervas,No,no-feat
1,2,Say So,Say So,Doja Cat,[Doja Cat],1,Doja Cat,No,no-feat
2,3,My Oh My,My Oh My (feat. DaBaby),Camila Cabello feat. DaBaby,"[Camila Cabello, DaBaby]",2,Camila Cabello,Yes,no-feat
3,3,My Oh My,My Oh My (feat. DaBaby),Camila Cabello feat. DaBaby,"[Camila Cabello, DaBaby]",2,DaBaby,Yes,no-feat
4,4,Moon,Moon (And It Went Like),Kid Francescoli,[Kid Francescoli],1,Kid Francescoli,No,no-feat
...,...,...,...,...,...,...,...,...,...
83,64,I Think I'm OKAY,I Think I'm OKAY (with YUNGBLUD & Travis Barker),"Machine Gun Kelly, YUNGBLUD, and Travis Barker","[Machine Gun Kelly, YUNGBLUD, Travis Barker]",3,Machine Gun Kelly,No,no-feat
84,64,I Think I'm OKAY,I Think I'm OKAY (with YUNGBLUD & Travis Barker),"Machine Gun Kelly, YUNGBLUD, and Travis Barker","[Machine Gun Kelly, YUNGBLUD, Travis Barker]",3,YUNGBLUD,No,no-feat
85,64,I Think I'm OKAY,I Think I'm OKAY (with YUNGBLUD & Travis Barker),"Machine Gun Kelly, YUNGBLUD, and Travis Barker","[Machine Gun Kelly, YUNGBLUD, Travis Barker]",3,Travis Barker,No,no-feat
86,65,Myself,Myself,Bazzi,[Bazzi],1,Bazzi,No,no-feat


In [46]:
# Export
df_songs_artists.to_csv('data/proj03-df-songs-data-artists.csv')

## Songs - TikTok Stats

This dataframe contains the statistics (number of posts made with the song) from TikTok.

In [47]:
# Dataframe
df_songs_tiktok_stats.head()

,index,song_id,song,sp_name,tiktok_stats,value,timestp,diff
0,0,1,Roxanne,ROXANNE,"{'value': 2100000, 'timestp': '2020-01-19T00:0...",2100000.0,2020-01-19T00:00:00.000Z,NaN
1,0,1,Roxanne,ROXANNE,"{'value': 2100000, 'timestp': '2020-01-20T00:0...",2100000.0,2020-01-20T00:00:00.000Z,0.0
2,0,1,Roxanne,ROXANNE,"{'value': 2100000, 'timestp': '2020-01-21T00:0...",2100000.0,2020-01-21T00:00:00.000Z,0.0
3,0,1,Roxanne,ROXANNE,"{'value': 2200000, 'timestp': '2020-01-28T00:0...",2200000.0,2020-01-28T00:00:00.000Z,100000.0
4,0,1,Roxanne,ROXANNE,"{'value': 2200000, 'timestp': '2020-01-29T00:0...",2200000.0,2020-01-29T00:00:00.000Z,0.0


In [48]:
# Metadata about the daframe
df_songs_tiktok_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10270 entries, 0 to 10269
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   index         10270 non-null  int64  
 1   song_id       10270 non-null  int64  
 2   song          10270 non-null  object 
 3   sp_name       10270 non-null  object 
 4   tiktok_stats  10264 non-null  object 
 5   value         10264 non-null  float64
 6   timestp       10264 non-null  object 
 7   diff          10204 non-null  float64
dtypes: float64(2), int64(2), object(4)
memory usage: 642.0+ KB


In [49]:
# Export
df_songs_tiktok_stats.to_csv('data/proj03-df-songs-data-tiktok-stats.csv')

## Artistst - General

In [50]:
# Final Dataframe
df_artists = df_cmc_artists.copy()

# Add artist ID
df_artists['artist_id'] = [index + 1 for index in df_artists.index]

# Reorder columns
cols = list(df_artists.columns)
cols = [cols[-1]] + cols[:-1]
df_artists = df_artists[cols]

# Check the result
df_artists

,artist_id,artist,sp_genres,sp_popularity,sp_followers,cmc_art_sp_city,cmc_art_tiktok_audience,cmc_code2,cmc_tags,cmc_sp_monthly_listeners,cm_artist_rank,sp_popularity_rank,sp_monthly_listeners_rank,youtube_daily_video_views_rank,youtube_monthly_video_views_rank,sp_where_people_listen
0,1,Arizona Zervas,"[pop rap, rap, rhode island rap]",80,486779,{'obj': {'Atlanta': [{'timestp': '2020-03-21T0...,{'obj': {'top_countries': [{'name': 'United St...,us,"[pop, pop rap]",15313268,353,459,215,660,562,"[{'name': 'Chicago', 'code2': 'us', 'listeners..."
1,2,Doja Cat,"[la indie, pop]",88,3445699,{'obj': {'Atlanta': [{'timestp': '2020-03-21T0...,{'obj': {'top_countries': [{'name': 'United St...,us,"[pop, pop rap, la indie, electropop]",31770199,61,101,54,94,100,"[{'name': 'Mexico City', 'code2': 'mx', 'liste..."
2,3,Camila Cabello,"[dance pop, pop, post-teen pop]",87,17989469,{'obj': {'Brisbane': [{'timestp': '2020-03-21T...,{'obj': {'top_countries': [{'name': 'United St...,us,"[pop, dance pop, post-teen pop]",36816546,54,122,33,124,118,"[{'name': 'Mexico City', 'code2': 'mx', 'liste..."
3,4,DaBaby,"[north carolina hip hop, rap]",95,4326193,{'obj': {'Atlanta': [{'timestp': '2020-03-21T0...,"{'obj': {'top_countries': [], 'audience_gender...",us,"[rap, nc hip hop]",41793864,41,13,26,101,91,"[{'name': 'Chicago', 'code2': 'us', 'listeners..."
4,5,Kid Francescoli,"[electronica, french indie pop, french indietr...",61,82296,{'obj': {'Berlin': [{'timestp': '2020-03-21T00...,"{'obj': {'top_countries': [], 'audience_gender...",fr,"[french indie pop, french indietronica, electr...",1316092,6839,6189,4546,0,0,"[{'name': 'Paris', 'code2': 'fr', 'listeners':..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,84,Machine Gun Kelly,"[ohio hip hop, pop rap, rap]",86,2454191,{'obj': {'Atlanta': [{'timestp': '2020-03-21T0...,{'obj': {'top_countries': [{'name': 'United St...,us,"[rap, hip hop, pop rap]",16127772,203,145,198,221,208,"[{'name': 'Chicago', 'code2': 'us', 'listeners..."
84,85,YUNGBLUD,"[british indie rock, modern alternative rock, ...",78,1085134,{'obj': {'Atlanta': [{'timestp': '2020-03-21T0...,{'obj': {'top_countries': [{'name': 'United St...,us,"[rock, british indie rock, modern rock, modern...",7878138,698,613,567,0,0,"[{'name': 'Chicago', 'code2': 'us', 'listeners..."
85,86,Travis Barker,[rap rock],77,252326,{'obj': {'Atlanta': [{'timestp': '2020-03-21T0...,"{'obj': {'top_countries': [], 'audience_gender...",us,"[pop rap, rap rock]",5576776,1214,704,881,1899,2447,"[{'name': 'Chicago', 'code2': 'us', 'listeners..."
86,87,Bazzi,"[pop, post-teen pop]",83,3671755,{'obj': {'Atlanta': [{'timestp': '2020-03-21T0...,{'obj': {'top_countries': [{'name': 'United St...,us,"[pop, dance pop, post-teen pop]",19578237,258,252,148,803,740,"[{'name': 'Chicago', 'code2': 'us', 'listeners..."


In [51]:
# Export
df_artists.to_csv('data/proj03-df-artists.csv')

# PostgreSQL

## Setup

In [52]:
load_dotenv(find_dotenv())

True

In [53]:
# Variables necessary
db_server = 'postgresql'
user = 'postgres'
password = os.getenv('pg_admin')
ip = 'localhost'
db_name = 'proj03-tiktok-viral-songs'

# Create the engine
engine = db.create_engine(f'{db_server}://{user}:{password}@{ip}/{db_name}')

# Open the connection
conn = engine.connect()

## Export the datasets

In [54]:
# Remove columns that are dictionaries
df_songs = df_songs.drop(columns=['cmc_tiktok_stats'])
df_songs_tiktok_stats = df_songs_tiktok_stats.drop(columns=['tiktok_stats'])

In [55]:
# Export
df_songs.to_sql(name='df-songs-gen', con=conn, if_exists='replace', index=False)
df_songs_genres.to_sql(name='df-songs-genres', con=conn, if_exists='replace', index=False)
df_songs_audio_features_summary.to_sql(name='df-songs-audio-features', con=conn, if_exists='replace', index=False)
df_songs_artists.to_sql(name='df-songs-artists', con=conn, if_exists='replace', index=False)
df_songs_tiktok_stats.to_sql(name='df-songs-tiktok-stats', con=conn, if_exists='replace', index=False)